# 1. Loading Data

In [1]:
from IPython.display import clear_output

def hint(message):
    """
    erase previous ipynb output and show new message
    """
    clear_output()
    print(message)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

hint("loading data...")
train = pd.read_csv('data/train.csv')
train, valid = train_test_split(train, test_size=0.2)

labels = [
    'toxic', 
    'severe_toxic', 
    'obscene', 
    'threat', 
    'insult', 
    'identity_hate'
]

Ytr = train[labels].values
Yva = valid[labels].values

hint('Label distribution between training and validation set:')
print(pd.DataFrame({
    'label': labels,
    'train': [np.mean(train[label]) for label in labels],
    'validation' : [np.mean(valid[label]) for label in labels],
}))

Label distribution between training and validation set:
           label     train  validation
0          toxic  0.095656    0.096600
1   severe_toxic  0.009878    0.010465
2        obscene  0.052500    0.054739
3         threat  0.002985    0.003039
4         insult  0.049210    0.049977
5  identity_hate  0.008664    0.009369


# 2. Pre-processing

## 2.1 Cleaning

Gather resources

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

nltk.download('wordnet')

lmtzr = WordNetLemmatizer()
eng_stopwords = (
    'what', 'which', 'who', 'whom', 
    'this', 'that', 'these', 'those', 
    'am', 'is', 'are', 'was', 'were', 
    'be', 'been', 'being', 
    'have', 'has', 'had', 'having', 
    'do', 'does', 'did', 'doing', 
    'a', 'an', 'the', 
    'and', 'but', 'if', 'or', 
    'because', 'as', 'until', 'while', 
    'of', 'at', 'by', 'for', 'with', 
    'about', 'against', 'between', 
    'into', 'through', 'during', 'before', 'after', 
    'above', 'below', 'to', 'from', 
    'up', 'down', 'in', 'out', 'on', 'off', 
    'over', 'under', 'again', 'further', 
    'then', 'once', 'here', 
    'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 
    'few', 'more', 'most', 'other', 'some', 
    'such', 'no', 'nor', 'not', 'only', 
    'own', 'same', 'so', 'than', 'too', 'very', 
    'can', 'will', 'just', 'don', 'should', 'now'
)
appos = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "i'd" : "I would",
    "i'd" : "I had",
    "i'll" : "I will",
    "i'm" : "I am",
    "isn't" : "is not",
    "it's" : "it is",
    "it'll":"it will",
    "i've" : "I have",
    "let's" : "let us",
    "mightn't" : "might not",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "we'd" : "we would",
    "we're" : "we are",
    "weren't" : "were not",
    "we've" : "we have",
    "what'll" : "what will",
    "what're" : "what are",
    "what's" : "what is",
    "what've" : "what have",
    "where's" : "where is",
    "who'd" : "who would",
    "who'll" : "who will",
    "who're" : "who are",
    "who's" : "who is",
    "who've" : "who have",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "'re": " are",
    "wasn't": "was not",
    "we'll":" will",
    "didn't": "did not"
}

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ChuanLi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Define cleaning method

In [4]:
from nltk.tokenize import TweetTokenizer
import re

tkzr = TweetTokenizer(preserve_case=False)

def preprocess(comment):
  
    # credit to the author of this post:
    # https://www.kaggle.com/jagangupta/stop-the-s-toxic-comments-eda

    # remove special format
    comment = re.sub('\n\t', '', comment)

    # remove IP addresses
    comment = re.sub('\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' specipaddress ', comment)

    # remove username
    comment = re.sub("\[\[User.*\]", ' specusername ', comment)
    comment = re.sub("\[\[User.*\|", ' specusername ', comment)

    # tokenization 
    tokens = tkzr.tokenize(comment)

    # aphostophe replacement
    tokens = [ appos[token] if token in appos else token for token in tokens]

    # remove stopwords
    tokens = [ token for token in tokens if not token in eng_stopwords ]

    # stemming
    tokens = [ lmtzr.lemmatize(token, 'v') for token in tokens]

    return " ".join(tokens)

Cleaning

In [5]:
hint('Cleaning train set...')
Xtr = train['comment_text'].apply(lambda c: preprocess(c))
hint('Cleaning test set...')
Xva = valid['comment_text'].apply(lambda c: preprocess(c))
hint('Done')

Done


## 2.2 Making Sequences

In [6]:
from keras.preprocessing import text as ktxt, sequence

vocab_max = 100000

hint('Fitting the tokenizer...')
tokenizer = ktxt.Tokenizer(num_words=vocab_max)
tokenizer.fit_on_texts(Xtr)

hint('Tokenizing...')
Xtr = tokenizer.texts_to_sequences(Xtr)
Xva = tokenizer.texts_to_sequences(Xva)

hint('padding the sequences...')
maxlen = 200  # padded/cropped comment length
Xtr = sequence.pad_sequences(Xtr, maxlen=maxlen)
Xva = sequence.pad_sequences(Xva, maxlen=maxlen)

hint('Done')

Done


## 2.3 Prepare Embedding Matrix

In [7]:
import csv

emb_file = 'preembedding/glove.6B.300d.txt'

hint("Loading pre-embedding file...")
emb = pd.read_table(emb_file, " ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

hint("Preparing embedding matrix...")
emb_size = 300
embedding_matrix = np.random.normal(
    emb.mean(axis=0), 
    emb.std(axis=0), 
    (vocab_max, emb_size)
)

hint("Done")

Done


In [8]:
hint("Constructing embedding matrix")
for word, i in tokenizer.word_index.items():
    if i < vocab_max and word in emb.index:
        embedding_matrix[i] = emb.loc[word].as_matrix()

hint("Done")

Done


In [9]:
emb = None

# 3. Model

## 3.1 Model Definition

In [12]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, GRU, SpatialDropout1D, Bidirectional
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.layers.convolutional import Conv1D, MaxPooling1D

x1 = x2 = x3 = model = None

# channel 1
input1 = Input(shape=(maxlen, ))
x1 = Embedding(vocab_max, emb_size, weights=[embedding_matrix])(input1)
x1 = SpatialDropout1D(0.5)(x1)
x1 = Conv1D(filters=256, kernel_size=2, activation='relu')(x1)
x1 = MaxPooling1D(pool_size=2)(x1)
x1 = SpatialDropout1D(0.2)(x1)
x1 = Bidirectional(GRU(64, return_sequences=True))(x1)
x1 = GlobalMaxPooling1D()(x1)

# channel 2
input2 = Input(shape=(maxlen, ))
x2 = Embedding(vocab_max, emb_size, weights=[embedding_matrix])(input2)
x2 = SpatialDropout1D(0.5)(x2)
x2 = Conv1D(filters=256, kernel_size=4, activation='relu')(x2)
x2 = MaxPooling1D(pool_size=2)(x2)
x2 = SpatialDropout1D(0.2)(x2)
x2 = Bidirectional(GRU(64, return_sequences=True))(x2)
x2 = GlobalMaxPooling1D()(x2)

# channel 3
input3 = Input(shape=(maxlen, ))
x3 = Embedding(vocab_max, emb_size, weights=[embedding_matrix])(input3)
x3 = SpatialDropout1D(0.5)(x3)
x3 = Conv1D(filters=256, kernel_size=8, activation='relu')(x3)
x3 = MaxPooling1D(pool_size=2)(x3)
x3 = SpatialDropout1D(0.2)(x3)
x3 = Bidirectional(GRU(64, return_sequences=True))(x3)
x3 = GlobalMaxPooling1D()(x3)

merged = concatenate([x1, x2, x3])

preds = Dense(6, activation="sigmoid")(merged)

model = Model([input1, input2, input3], preds)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## 3.2 Training

In [13]:
epochs = 5
batch_size = 128

def get_class_weight(x):
    k = 100
    return 3.32*np.log(k/x + 1)
    

history = model.fit(
    [Xtr, Xtr, Xtr], Ytr, 
    epochs=epochs, 
    batch_size=batch_size,
    validation_data=([Xva, Xva, Xva], Yva),
    class_weight={
        0: get_class_weight(98),
        1: get_class_weight(10),
        2: get_class_weight(53),
        3: get_class_weight(2),
        4: get_class_weight(49),
        5: get_class_weight(8),
    }
)

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 518s 4ms/step - loss: 0.1690 - acc: 0.9768 - val_loss: 0.0558 - val_acc: 0.9798
Epoch 2/5
127656/127656 [==============================] - 515s 4ms/step - loss: 0.1287 - acc: 0.9810 - val_loss: 0.0531 - val_acc: 0.9809
Epoch 3/5
127656/127656 [==============================] - 514s 4ms/step - loss: 0.1137 - acc: 0.9828 - val_loss: 0.0526 - val_acc: 0.9808
Epoch 4/5
 24576/127656 [====>.........................] - ETA: 6:36 - loss: 0.0993 - acc: 0.9848

KeyboardInterrupt: 